In [ ]:
# feature vectors
%pip install gdown
import gdown

url = "https://drive.google.com/drive/folders/1U5lsvfh8zSOfPatqXvdGHLo4j_XkEyw7?usp=share_link"
gdown.download_folder(url, quiet=True)

In [ ]:
# 128D vectors
%pip install gdown
import gdown

url = "https://drive.google.com/drive/folders/1Wjkc9I6N-InDznjsu5ETmtAecPSLQYQQ?usp=share_link"
gdown.download_folder(url, quiet=True)

In [3]:
import os
import zipfile
import numpy as np
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split, cross_val_score, KFold
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report, accuracy_score

def train_evaluate_svm(X_train, X_test, y_train, y_test):

  # standardize the features
  scaler = StandardScaler()
  X_train = scaler.fit_transform(X_train)
  X_test = scaler.transform(X_test)

  # train the SVM classifier
  svm_model = SVC(kernel='linear', random_state=42)
  #svm_model = SVC(kernel='poly', random_state=42)
  print('model loaded')
  # cross-validation sul training set
  cv = KFold(n_splits=5, shuffle=True, random_state=42)
  cv_scores = cross_val_score(svm_model, X_train, y_train, cv=cv, scoring='accuracy', n_jobs=-1)
  print(cv_scores)

  svm_model.fit(X_train, y_train)

  # make predictions and evaluate
  y_pred = svm_model.predict(X_test)

  # accuracy
  accuracy = accuracy_score(y_test, y_pred)
  class_report = classification_report(y_test, y_pred)
  print(accuracy)
  print(class_report)

def main():
  folder_path = "/content/feature_vectors"
  #folder_path = "/content/128D_vectors"
  for file_name in os.listdir(folder_path):

    # zipped files in the folder biometric_systems_dataset
    if file_name.endswith(".zip"):

      # full path to the zip file
      file_path = os.path.join(folder_path, file_name)

      # create folder in the same path with the name of the zip file
      extract_folder = os.path.join(folder_path, file_name.replace(".zip", ""))
      os.makedirs(extract_folder, exist_ok=True)

      # open and extract the file
      with zipfile.ZipFile(file_path, mode="r") as zip_ref:
        zip_ref.extractall(extract_folder)

  X_train_eyes, y_train_eyes = [], []
  X_test_eyes, y_test_eyes = [], []
  X_train_mouth, y_train_mouth = [], []
  X_test_mouth, y_test_mouth = [], []
  X_train_nose, y_train_nose = [], []
  X_test_nose, y_test_nose = [], []
  sets = ["train", "test"]
  for subfolder in os.listdir(folder_path):
    if subfolder.endswith(".zip"):
      continue
    for s in sets:
      subfolder_path = os.path.join(folder_path, subfolder, s)
      for file_name in os.listdir(subfolder_path):
        file_path = os.path.join(subfolder_path, file_name)
        embedding = np.load(file_path)
        feature, age, gender, race, date_dime = file_name.split("_")
        if s == "train":
          if subfolder == "eyes":
            X_train_eyes.append(embedding)
            y_train_eyes.append(int(race))
          elif subfolder == "mouth":
            X_train_mouth.append(embedding)
            y_train_mouth.append(int(race))
          elif subfolder == "nose":
            X_train_nose.append(embedding)
            y_train_nose.append(int(race))
        else:
          if subfolder == "eyes":
            X_test_eyes.append(embedding)
            y_test_eyes.append(int(race))
          elif subfolder == "mouth":
            X_test_mouth.append(embedding)
            y_test_mouth.append(int(race))
          elif subfolder == "nose":
            X_test_nose.append(embedding)
            y_test_nose.append(int(race))

  X_train_eyes, y_train_eyes = np.array(X_train_eyes), np.array(y_train_eyes)
  X_test_eyes, y_test_eyes = np.array(X_test_eyes), np.array(y_test_eyes)
  X_train_mouth, y_train_mouth = np.array(X_train_mouth), np.array(y_train_mouth)
  X_test_mouth, y_test_mouth = np.array(X_test_mouth), np.array(y_test_mouth)
  X_train_nose, y_train_nose = np.array(X_train_nose), np.array(y_train_nose)
  X_test_nose, y_test_nose = np.array(X_test_nose), np.array(y_test_nose)

  X_all = [[X_train_eyes,X_test_eyes], [X_train_mouth, X_test_mouth], [X_train_nose, X_test_nose]]
  y_all = [[y_train_eyes, y_test_eyes], [y_train_mouth, y_test_mouth], [y_train_nose, y_test_nose]]

  for idx, (X, y) in enumerate(zip(X_all, y_all)):
    X_train, X_test = X
    y_train, y_test = y
    if idx == 0:
      train_evaluate_svm(X_train, X_test, y_train, y_test)

if __name__ == "__main__":
  main()

model loaded


KeyboardInterrupt: 